In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import xarray as xr
from scipy import stats
from scipy.stats import linregress
import os
import glob
from pathlib import Path  
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.signal import correlate 
import natsort 
import datetime as dt
import scipy
import utide
from astropy.io import ascii
import cartopy.crs as ccrs
import pandas as pd
import matplotlib.lines as mlines
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os
from scipy.signal import medfilt
import matplotlib.patches as mpatches
import os

In [2]:
df = pd.read_csv("datafiles/INCOIS_Readme.txt", delimiter=r"\s+")
df=df[:-1]
df.head()

,Station,Longitude,Latitude
0,AdaniHazira,72.616,21.100
1,AerialBay,93.033,13.283
2,Beypore,75.808,11.171
3,CampbellBay,93.933,7.000
4,CarNicobar,92.776,9.234


In [4]:
paths=[]
stations=[]
directory_path= "outputs"
sub_files=os.listdir(directory_path)
sub_files=natsort.natsorted(sub_files,reverse=False)
for j in sub_files:
    if '5min' in j:
        full_path = os.path.join(directory_path, j)
        parts = full_path.split('/')  # Split the string by the slash character
        station_year = parts[-1].replace('_5min_interpolated.out', '')
            
        # stations.append(station_year)
        paths.append(full_path)


In [5]:
station_name = paths[3][8:-27]
station_year = paths[3][-26:-22]
station_year

'2018'

In [6]:
df = pd.read_csv(paths[0], delimiter=r"\s+")
df

,2015-12-08 00:00:00,3960.0
0,2015-12-08 00:05:00,3910.0
1,2015-12-08 00:10:00,3870.0
2,2015-12-08 00:15:00,3810.0
3,2015-12-08 00:20:00,3740.0
4,2015-12-08 00:25:00,3700.0
...,...,...
6906,2015-12-31 23:35:00,5110.0
6907,2015-12-31 23:40:00,5200.0
6908,2015-12-31 23:45:00,5290.0
6909,2015-12-31 23:50:00,5350.0


In [12]:

def create_blank_year_file(year, template_path, max_gap, base_name):
    start_of_year = pd.Timestamp(f"{year}-01-01 00:00:00")
    end_of_year = pd.Timestamp(f"{year}-12-31 23:59:59")
    full_year_index = pd.date_range(start=start_of_year, end=end_of_year, freq='1T')
    
    # Create a blank DataFrame with 'Time' and 'Sl' columns
    df_blank = pd.DataFrame(full_year_index, columns=['Time'])
    df_blank['Sl'] = np.nan

    # Construct the new file name
    new_file_name = f'{base_name}_{year}_5min_interpolated.csv'

    # Replace the original file name in the path with the new file name
    output_file_path = template_path.replace(template_path.split('/')[-1], new_file_name)

    # Save to CSV without the index
    df_blank.to_csv(output_file_path, index=False)

def process_file(file_path, max_gap, station_year):
    # Read CSV
    df = pd.read_csv(file_path, delimiter=r"\s+", names=["Time", "Sl"], header=None)
    df['Sl'].replace("None", np.nan, inplace=True)

    # Convert 'Sl' to float with error handling
    df['Sl'] = pd.to_numeric(df['Sl'], errors='coerce')

    # Convert 'Time' to datetime format and set as index
    df["Time"] = pd.to_datetime(df["Time"], errors='coerce')
    df = df.set_index('Time')

    # Resample time in 1-minute intervals without interpolation
    df = df.resample("1T").asfreq()
    df['Sl'] = df['Sl'].interpolate(method='linear', limit=4)

    # Create a full year index and reindex
    start_of_year = pd.Timestamp(f"{station_year}-01-01 00:00:00")
    end_of_year = pd.Timestamp(f"{station_year}-12-31 23:59:00")
    full_year_index = pd.date_range(start=start_of_year, end=end_of_year, freq='1T')
    df = df.reindex(full_year_index)

    # Reset the index to make 'Time' a column again
    df = df.reset_index().rename(columns={'index': 'Time'})

    # Save the modified DataFrame without the index
    output_file_path = file_path.replace('.out', '.csv')
    df.to_csv(output_file_path, index=False)

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Define the maximum gap to fill
max_gap = 5
max_year = 2022

# Initialize the previous year tracker
prev_year = None

# Process each file in the paths array
for file_path in paths:
    # Extract the station year from the file path
    station_name = file_path[8:-27]
    station_year = int(file_path[-26:-22])

    # Check for missing years and create files for them
    if prev_year is not None and station_year > prev_year + 1:
        for missing_year in range(prev_year + 1, min(station_year, max_year + 1)):
            create_blank_year_file(missing_year, file_path, max_gap,station_name)

    # Update the previous year tracker
    prev_year = station_year

    # Process the current file
    process_file(file_path, max_gap,station_year)
